# Kinetics tutorial


$$\frac{\mathrm d[\mathrm A]}{\mathrm dt} = -k[\mathrm A]^n \qquad [\mathrm A]_0 = c_0$$


$$c_{\mathrm A}(t) = \sqrt[1-n]{c_0^{1-n} + kt(n-1)}$$


special cases

0 order, $n = 0$: $$c(t)=c_0 - kt$$

1$^{\mathrm{st}}$ order, $n = 1$: $$c(t)=c_0e^{-kt}$$

2$^{\mathrm{nd}}$ order, $n = 2$: $$c(t)=\frac{c_0}{1 + c_0kt}$$

with this initial condition 


In [1]:
%matplotlib notebook
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interactive
from IPython import display
import numpy as np


import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


fig, ax = plt.subplots()
x_max = 2
ax.set_xlim(-0.02, x_max)
ax.set_ylim(-0.02, 1)
# ax.set_title('beat frequencies')
ax.set_ylabel('$c_A/c_0$')
ax.set_xlabel('Time')
plt_A, = ax.plot([], [], color='r', label='n$^{th}$ order')
plt_0, = ax.plot([], [], color='b', label='Zero order', lw=0.5)
plt_1, = ax.plot([], [], color='g', label='1$^{st}$ order', lw=0.5)
plt_2, = ax.plot([], [], color='black', label='2$^{nd}$ order', lw=0.5)
ax.legend()

times = np.linspace(0,x_max,500)

def A_to_B(c0, k, n):

    # for n == 1, it is just simple exponencial decay
    if n == 1:
        y_data = np.exp(-k * times)
    else:
        # expression inside the root
        expr_in_root = c0 ** (1 - n) + k * (n - 1) * times
        # we have to set all negative values to 0 (this is important for n < 1),
        # because the root of negative value would cause error
        expr_in_root = expr_in_root.clip(min=0)
        y_data = np.power(expr_in_root, 1.0/(1-n)) / c0
    
    plt_A.set_data(times, y_data)
    plt_0.set_data(times, [0 if num < 0 else num for num in (c0 - k*times) / c0])
    plt_1.set_data(times, np.exp(-k * times))
    plt_2.set_data(times, 1/(1 + c0*k*times))
    
    

interactive(A_to_B, c0=(0.1,50.0), k=(0.1,5.0), n=(0.00, 4.00))

<IPython.core.display.Javascript object>

interactive(children=(FloatSlider(value=25.05, description='c0', max=50.0, min=0.1), FloatSlider(value=2.55000…

# Kinetics of photobleaching
How to correctly determine quantum yield

Rate of compound decay is directly proportional to the intensity of incident light $I_0$, quantum yield of the photoreaction $\Phi$ and fraction of light that was absorbed by the solution: $1 - T(t)$, where $T(t)$ is transmitance at time $t$.

$$\frac{\mathrm dc(t)}{\mathrm dt} = -\frac{1}{V}I_0\Phi\left(1 - T(t)\right) = -k\left(1 - 10^{-A(t)}\right)$$

where $A(t) = \varepsilon c(t)l$ and $k = \frac{1}{V}I_0\Phi$

initial condition:
$$c(0) = c_0 = \frac{A_0}{\varepsilon l}$$

Integrated form:

$$c(t) = \frac{1}{\varepsilon l\ln 10}\ln\left(  10^{-\varepsilon lkt}  \left( 10^{\varepsilon c_0l} + 10^{\varepsilon lkt} - 1  \right)\right)$$

To derive equation for $A(t)$ instead of $c(t)$, we get rid of $\varepsilon l$ by using $A_0$ and use this substitution:

$$k^{\prime} = \frac{k}{c_0}$$

$$A(t) = \frac{1}{\ln 10}\ln\left(  10^{-A_0k^{\prime}t}  \left( 10^{A_0} + 10^{A_0k^{\prime}t} - 1  \right)\right)$$


Simplifications, for $A_0 >> 1$, we can approximate the absorbance term for short times as, differential equation changes to, because $10^{-A(t)}\approx0$ for short times :

$$\frac{\mathrm dc(t)}{\mathrm dt} = -k$$

$$c(t) = c_0 - kt$$
by substituting with $k^{\prime}$ and dividing by $c_0$ to get the ratio, we get simply
$$\frac{c(t)}{c_0} = 1 - k^{\prime}t$$

In other words, all light is absorbed by the molecules which leads to zero order rate law.


On the other hand, for very low initial absorbances, we can simplyfy the term by taking only 1 term in taylor series of exponential, so the $e^{-x} \approx 1 - x$ for $x\approx 0$. Therefore the term $10^{-A(t)}$ become $1 - \varepsilon c(t) l \ln 10$ and the differential law becomes first order decay.

$$\frac{\mathrm dc(t)}{\mathrm dt} = -k\varepsilon c(t) l \ln 10$$

After integration, we get this ratio, which holds only for very small initial absorbance, so the solution barely absorbs.

$$\frac{c(t)}{c_0} = 10^{-A_0k^{\prime}t}$$



It is better to have absorbance at least 2 to measure quantum yield, because the photobleaching is linear in time and....

If we cannot have solution with A > 2, we have to at least keep the conversion < 10 %, where the error will be minimal, even for very low initial absorbance $A_0$


In [2]:
import numpy as np
import plotly as py
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display, clear_output, Image

py.offline.init_notebook_mode(connected=True)

times = np.linspace(0, 2, num=1000)

layout = go.Layout(
    title='',
    xaxis=dict(
        title="Time",
        range=[-0.02, 2],
        showline=True,
        mirror='ticks',
        zeroline=False
    ),
    yaxis=dict(
        title="$c/c_0 = A/A_0$",
        range=[-0.02,1.02],
        showline=True,
        mirror='ticks',
        zeroline=False
    )
)

kp_wid = widgets.FloatSlider(min=0.1, max=20, value=1, description='k\'')
A0_wid = widgets.FloatLogSlider(base=10, min=-2, max=1, value=1, step=0.01, description='A0')

fig = go.FigureWidget(data=[
    dict(
        type='scatter',
        x=times,
        name='Integrated',
        mode='lines',
        line=dict(
            color='red',
            width=3
            )
        ),
    dict(
        type='scatter',
        x=times,
        name='1st order approx.',
        mode='lines',
        line=dict(
            color='black',
            width=1,
            dash='dash'
            )
        ),
    dict(
        type='scatter',
        x=times,
        name='Zero order approx.',
        mode='lines',
        line=dict(
            color='blue',
            width=1,
            dash='dot'
            )
        ),
], layout=layout)

# kp = k' = k / c0
def update(kp, A0):
    
    ln10 = np.log(10)
    
    A = np.log(10**(-A0*kp*times) * (10**A0 + 10**(A0*kp*times) - 1)) / ln10
    
    fig.data[0].y = A / A0
    fig.data[1].y = np.exp(-A0 * kp * ln10 * times)
    fig.data[2].y = 1 - kp * times

widgets.interactive(update, kp=kp_wid, A0=A0_wid)
display(widgets.HBox([A0_wid, kp_wid]))
display(fig)

update(kp_wid.value, A0_wid.value)

FigureWidget({
    'data': [{'line': {'color': 'red', 'width': 3},
              'mode': 'lines',
            …